In [ ]:
# Importing libraries
import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.graph_objects as go
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point
import plotly.express as px
import re
import time
alt.data_transformers.enable("vegafusion")
alt.data_transformers.disable_max_rows()

In [ ]:
df = pd.read_csv("/Users/jaigollapudi/Downloads/EV-Visualization/data/Electric_Vehicle_Population_Data.csv")

# Filtering the DataFrame to include only rows where the 'State' column is 'WA'
df = df[df['State'] == 'WA']

## Visualization 1 - Map Chart

In [ ]:
# Sample data extraction function
def extract_coordinates(point_str):
    if isinstance(point_str, str):
        # Remove 'POINT (' and ')' and split by space
        coords = point_str.replace('POINT (', '').replace(')', '').split()
        return float(coords[1]), float(coords[0])  # Note: coords are in order of latitude, longitude
    return None, None

# Assuming df is your DataFrame and 'Vehicle Location' contains the Point data
df['Latitude'], df['Longitude'] = zip(*df['Vehicle Location'].apply(extract_coordinates))
df = df.dropna(subset=['Latitude', 'Longitude'])

# Create a new Folium map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the cluster instead of the map
for idx, row in df.iterrows():
    popup_content = f"{row['Make']} {row['Model']} ({row['Model Year']})"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_content
    ).add_to(marker_cluster)

In [ ]:
# Save the map to an HTML file
m.save('map_chart.html')

## Visualization 2 - Line Chart

In [ ]:
# Converting 'Model Year' to a string to prevent it from being treated as a continuous quantity
df['Model Year'] = df['Model Year'].astype(str)

# Aggregating the data to count registrations per year per EV type
agg_data = df.groupby(['Model Year', 'Electric Vehicle Type']).size().reset_index(name='Registrations')

# Creating the line chart
line_chart = alt.Chart(agg_data).mark_line(point=True).encode(
    x=alt.X('Model Year:O', axis=alt.Axis(title='Model Year')),
    y=alt.Y('Registrations:Q', axis=alt.Axis(title='Number of Registrations')),
    color='Electric Vehicle Type:N',
    tooltip=['Model Year', 'Electric Vehicle Type', 'Registrations']
).interactive().properties(
    title='Trend of Electric Vehicle Registrations by Type',
    width=400,
    height=300
)


# Outputting chart
line_chart

In [ ]:
# Saving chart to html
line_chart.save('line_chart.html')

## Visualization 3 - Stacked Bar Chart

In [ ]:
# Group and aggregate the data to count makes within each vehicle type
make_counts = df.groupby(['Make', 'Electric Vehicle Type']).size().reset_index(name='Count')

# Calculate the total count for each make regardless of the vehicle type
total_counts = make_counts.groupby('Make')['Count'].sum().reset_index()

# Get the top 10 makes by total count
top_makes = total_counts.nlargest(10, 'Count')['Make']

# Filter the original make_counts to only include the top 10 makes
top_make_counts = make_counts[make_counts['Make'].isin(top_makes)]

# Create the stacked bar chart with 'Electric Vehicle Type' on the x-axis
stacked_bar_chart = alt.Chart(top_make_counts).mark_bar().encode(
    x=alt.X('Electric Vehicle Type:N', title='Electric Vehicle Type'),
    y=alt.Y('sum(Count):Q', title='Number of Vehicles', stack='zero'),
    color=alt.Color('Make:N', 
                    title='Make',
                    scale=alt.Scale(scheme='tableau20')),  # Use a 20-color scheme for distinction
    order=alt.Order('sum(Count):Q', sort='descending'),  # Order the segments by size
    tooltip=[alt.Tooltip('Make:N', title='Make'),
             alt.Tooltip('Electric Vehicle Type:N', title='Vehicle Type'),
             alt.Tooltip('sum(Count):Q', title='Number of Vehicles')]
).properties(
    width=600,
    height=400,
    title='Market Share by Make within Electric Vehicle Types'
)

stacked_bar_chart

In [ ]:
# Saving chart to html
stacked_bar_chart.save('stacked_bar_chart.html')

## Visualization 4 - Bar Chart

In [ ]:
counts = df['Electric Utility'].value_counts().reset_index()
counts.columns = ['Electric Utility', 'Count']  # Rename columns appropriately
counts = counts.sort_values(by='Count', ascending=False).head(10)  # Ensure it's sorted in descending order

bar_chart = alt.Chart(counts).mark_bar().encode(
    x=alt.X('Electric Utility:N', sort='-y'),  # Specify nominal data and sort bars based on the counts in descending order
    y=alt.Y('Count:Q'),  # Quantitative scale for counts
    color=alt.Color('Count:Q'),
    tooltip=['Electric Utility', 'Count']  # Optional: show tooltip on hover
).properties(
    width=800,  # Adjust the width as needed
    height=400,  # Adjust the height as needed
    title='Top 10 Electric Utility Firms Used in WA')

bar_chart

In [ ]:
# Saving chart to html
bar_chart.save('bar_chart.png')

## Visualization 5